In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
import types
# Thisisabitofmagictomakematplotlibfiguresappearinlineinthenotebook
# ratherthaninanewwindow.
%matplotlib inline
# editthislinetochangethefiguresize
plt.rcParams['figure.figsize'] = (16.0, 10.0)
plt.rcParams['font.size'] = 16
# forceauto-reloadofimportmodulesbeforerunningcode
%load_ext autoreload
%autoreload 2


## Ζήτημα 1.1:

In [ ]:
!curl -Ohttp://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!mkdir -pdata && tar-xzvfcifar-10-python.tar.gz--directorydata

Usage: curl [options...] <url>
 -d, --data <data>          HTTP POST data
 -f, --fail                 Fail silently (no output at all) on HTTP errors
 -h, --help <category>      Get help for commands
 -i, --include              Include protocol response headers in the output
 -o, --output <file>        Write to file instead of stdout
 -O, --remote-name          Write output to a file named as the remote file
 -s, --silent               Silent mode
 -T, --upload-file <file>   Transfer local FILE to destination
 -u, --user <user:password> Server user and password
 -A, --user-agent <name>    Send User-Agent <name> to server
 -v, --verbose              Make the operation more talkative
 -V, --version              Show version number and quit

This is not the full help, this menu is stripped into categories.
Use "--help category" to get an overview of all categories.
For all options use the manual or "--help all".
mkdir: invalid option -- 'd'
Try 'mkdir --help' for more information.


### data_util.py

In [ ]:
import pickle
import os

def load_CIFAR_batch(filename):
  """
  Load a batch of images from the CIFAR10 python dataset
  """
  fh = open(filename, 'rb')
  data_dict = pickle.load(fh, encoding='latin1')
  X = data_dict['data'].reshape(10000,3,32,32).transpose(0,2,3,1)
  X = X.astype("float")/255.0
  Y = np.array(data_dict['labels'])
  fh.close()
  return X, Y

def load_CIFAR10(data_dir):
  """
  Load entire CIFAR10 python dataset
  """
  X_list = []
  Y_list = []
  for b in range(1,6):
    filename = os.path.join(data_dir, 'data_batch_%d' % (b, ))
    X_b, Y_b = load_CIFAR_batch(filename)
    X_list.append(X_b)
    Y_list.append(Y_b)
  X_train = np.concatenate(X_list)
  Y_train = np.concatenate(Y_list)
  X_test, Y_test = load_CIFAR_batch(os.path.join(data_dir, 'test_batch'))
  return X_train, Y_train, X_test, Y_test

def get_CIFAR10_data(num_train=49000, num_valid=1000, num_test=1000):
  """
  Load CIFAR10 dataset and assign train, test and val splits
  (total training data = 50k, test = 10k)
  """
  data_dir = r'C:\Users\mayts\Desktop\cifar-10-batches-py'
  X_train, Y_train, X_test, Y_test = load_CIFAR10(data_dir)

  X_val = X_train[num_train:(num_train+num_valid)]
  Y_val = Y_train[num_train:(num_train+num_valid)]
  X_train = X_train[0:num_train]
  Y_train = Y_train[0:num_train]
  X_test = X_test[0:num_test]
  Y_test = Y_test[0:num_test]

  return X_train, Y_train, X_val, Y_val, X_test, Y_test

# allow accessing these functions by data_util.*
data_util=types.SimpleNamespace()
data_util.load_CIFAR_batch=load_CIFAR_batch
data_util.load_CIFAR10=load_CIFAR10
data_util.get_CIFAR10_data=get_CIFAR10_data

### im_util.py

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def remove_ticks(ax):
  """
  Remove axes tick labels
  """
  ax.set_xticklabels([])
  ax.set_yticklabels([])
  ax.set_xticks([])
  ax.set_yticks([])

def plot_classification_examples(Y_hat,Y_test,im,names):
  """
  Plot sample images with predictions Y_hat and true labels Y_test
  """
  fh = plt.figure()
  num_test=Y_test.size
  for i in range(10):
    r = np.random.randint(num_test)
    ax=plt.subplot(1,10,i+1)
    remove_ticks(ax)
    lh=plt.xlabel(names[Y_hat[r]])
    if (Y_hat[r]==Y_test[r]):
      lh.set_color('green')
    else:
      lh.set_color('red')
    plt.imshow(im[r])

def plot_weights(W, names):
  """
  Plot images for each weight vector in W
  """
  fh = plt.figure()
  for i in range(10):
    W_im = np.reshape(W[:,i],(32,32,3))
    W_im = normalise_01(W_im)
    ax=plt.subplot(1,10,i+1)
    remove_ticks(ax)
    plt.xlabel(names[i])
    plt.imshow(W_im)

def normalise_01(im):
  """
  Normalise image to the range (0,1)
  """
  mx = im.max()
  mn = im.min()
  den = mx-mn
  small_val = 1e-9
  if (den < small_val):
    print('image normalise_01 -- divisor is very small')
    den = small_val
  return (im-mn)/den

# allow accessing these functions by im_util.*
im_util=types.SimpleNamespace()
im_util.remove_ticks=remove_ticks
im_util.plot_classification_examples=plot_classification_examples
im_util.plot_weights=plot_weights
im_util.normalise_01=normalise_01

In [ ]:
"""Load CIFAR10 data"""


num_classes=10
num_dims=32*32*3

cifar10_names=['airplane', 'automobile','bird','cat','deer','dog','frog','horse','ship','truck']

num_train=49000
num_valid=1000
num_test=10000

im_train,Y_train,im_valid,Y_valid,im_test,Y_test = data_util.get_CIFAR10_data(num_train,num_valid,num_test)

X_train=np.reshape(im_train,(num_train,num_dims))
X_valid=np.reshape(im_valid,(num_valid,num_dims))
X_test=np.reshape(im_test,(num_test,num_dims))

print('X_train shape = ',X_train.shape)  # 3072 = num_dims

### Οπτικοποίηση μέσης εικόνας κάθε κατηγορίας
Για το κομμάτι κώδικα που έπρεπε να συμπληρώσουμε. Σύνοψη:Υπολόγισα τη μέση εικόνα για κάθε τάξη, επαναλήφθηκε σε κάθε συγκεκριμένη κατηγορία και βρήκα τα δείγματα εικόνων που ανήκουν σε αυτήν την κατηγορία
τελικά μέσος όρος των εικόνων κατά μήκος του άξονα 0 και προσάρτηση του προκύπτοντος στη λίστα avg_im.

In [ ]:
"""Visualise average image"""

avg_im = []

# base code: use first image of each class
for i in range(10):
    j = next(k for k in range(num_train) if Y_train[k]==i)
    avg_im.append(im_train[j])

""" Remove the first 10 images from avg_im. This step is necessary because the first 10 images are incorrectly added
to the avg_im array due to the execution of the algorithm that computes the average image for each class."""
avg_im = avg_im[10:]

# the np.where, np.mean, im_train ... is from library NumPy
# Compute the average image for each class and store in avg_im
for i in range(num_classes):
  #Finds the sample images that match the current class.
    idx = np.where(Y_train == i)[0]
    #Calculates the average of the images for the current class and stores it in the avg_im array.
    avg_im.append(np.mean(im_train[idx], axis=0))

for i in range(10):
    ax=plt.subplot(1,10,i+1)
    im_util.remove_ticks(ax)
    plt.xlabel(cifar10_names[i])
    plt.imshow(avg_im[i])

## Ζήτημα 1.2:

Στην ερώτηση 1.2, υπολογίζουμε πρώτα τον μέσο όρο των εικόνων για κάθε τάξη, με βάση το σετ εκπαίδευσης. Στη συνέχεια, για κάθε εικόνα στο σύνολο δοκιμής, υπολογίζουμε το τετράγωνο της απόστασης μεταξύ αυτής και της μέσης εικόνας κάθε κατηγορίας. Τέλος, προβλέπουμε την κλάση εικόνας ως αυτή με τη μικρότερη απόσταση. Αυτή η λειτουργία επιτρέπει την κατηγοριοποίηση των δοκιμαστικών εικόνων με βάση τον μέσο όρο των εικόνων κάθε κατηγορίας.

In [ ]:
"""Nearest Mean Classifier"""

#FORNOW: random labels
Y_hat=np.random.randint(0,10,num_test)


# Compute the squared distance between two images
def squared_distance(image1, image2):
    return np.sum((image1 - image2) ** 2)

# Nearest Mean Classifier
"""
*****************************************************
"""
Y_hat = np.zeros(num_test, dtype=int)

for i in range(num_test):
    # Calculation of the distances from the average values ​​of the classes
    distances = [squared_distance(im_test[i], avg_im[j]) for j in range(num_classes)]

    # Find the class with the minimum distance and store it in Y_hat
    Y_hat[i] = np.argmin(distances)

nm_accuracy=np.sum(Y_hat==Y_test)/num_test
im_util.plot_classification_examples(Y_hat.astype(int),Y_test,im_test,cifar10_names)

print('Nearest mean classifier accuracy = %.2f%%' % (100.0*nm_accuracy))

## Ζήτημα 1.3:

In [ ]:
"""Nearest Neighbour Classifier"""
def compute_distances(desc1, desc2):
    """
    Compute Euclidean distances between descriptors

    Inputs: desc1=descriptor array (N1, num_dims)
            desc2=descriptor array (N2, num_dims)

    Returns: dists=array of distances (N1,N2)
    """
    N1,num_dims=desc1.shape
    N2,num_dims=desc2.shape

    ATB=np.dot(desc1,desc2.T)
    AA=np.sum(desc1*desc1,1)
    BB=np.sum(desc2*desc2,1)

    dists = -2*ATB + np.expand_dims(AA,1) + BB

    return dists


num_test_small=1000
X_test_small=X_test[0:num_test_small]
Y_test_small=Y_test[0:num_test_small]

Y_hat=np.random.randint(0,10,num_test_small)

"""
*****************************************************
"""
distances = compute_distances(X_test_small, X_train)
for i in range(num_test_small):
  # We match the class of the test image to the class of the closest training image.
    Y_hat[i] = Y_train[np.argmin(distances[i])]


nn_accuracy=np.sum(Y_hat==Y_test_small)/num_test_small
print('Nearest neighbour classifier accuracy =% .2f%%' % (100.0*nn_accuracy))

### Σχολιασμός αποτελέσματος Nearest neighbor σε σχέση με Nearest Mean Classifier

Ενώ ο πλησιέστερος μέσος ταξινομητής εστιάζει αποκλειστικά στην ταξινόμηση νέων παραδειγμάτων με βάση τις αποστάσεις από τον μέσο όρο κάθε τάξης εκπαίδευσης, αγνοώντας τη διακύμανση και το σχήμα των πραγματικών κατανομών δεδομένων, η προσέγγιση του Πλησιέστερου Γείτονα εξετάζει τα πλήρη δεδομένα εκπαίδευσης λεπτομερώς για να κάνει προβλέψεις. Λαμβάνοντας υπόψη όλες τις διαθέσιμες πληροφορίες από κάθε σημείο εκπαίδευσης και όχι απλώς τους μέσους όρους, το Nearest Neighbor είναι καλύτερα σε θέση να αντικατοπτρίζει σύνθετες, διαφοροποιημένες σχέσεις στον χώρο χαρακτηριστικών και ως εκ τούτου να δημιουργεί πιο ακριβείς, αντιπροσωπευτικές αποφάσεις που ευθυγραμμίζονται πιο στενά με την πραγματική δομή των δεδομένων . Μέσω ενός πιο ολοκληρωμένου χειρισμού του πλήρους χαρακτήρα του δείγματος εκπαίδευσης, το Nearest Neighbor ξεπερνά την περιορισμένη, αφηρημένη προβολή του Nearest Mean και παρέχει ως αποτέλεσμα ανώτερη προγνωστική ακρίβεια.

## Ζήτημα 1.4:

In [ ]:
"""Linear Classifier"""


def one_hot(Y, num_classes):
    """convert class labels to one-hot vector"""
    num_train=Y.size
    T = np.zeros((num_train, num_classes))
    T[np.arange(num_train), Y]=1
    return T

"""
*****************************************************
"""
# Fit a linear classifier to the CIFAR10 data
# Convert the class labels to binary
T_train = one_hot(Y_train, num_classes)
# We apply linear regression to train the classifier
# We use the lstsq method of numpy to find the weights W of the classifier as we have been asked by the statement of the exercise
W, _, _, _ = np.linalg.lstsq(X_train, T_train, rcond=None)



# predict labels on the test set using W

T_hat = np.dot(X_test,W)
Y_hat = np.argmax(T_hat,1)

lin_accuracy=np.sum(Y_hat==Y_test)/num_test

print('Linear classifier accuracy =% .2f%%' % (100.0*lin_accuracy))

# visualise the linear weights
im_util.plot_weights(W, cifar10_names)

"Σχολιασμός οπτικοποίησης βαρών..."

Οπτικοποίηση των βαρών του γραμμικού ταξινομητή ως εικόνες. Τα βάρη που μαθαίνονται από τον γραμμικό ταξινομητή μπορούν να απεικονιστούν ως μοτίβα που αντιπροσωπεύουν κάθε τάξη. Τα μοτίβα δείχνουν τις σημαντικές περιοχές των εικόνων που χρησιμοποιεί ο ταξινομητής για να κάνει τις προβλέψεις του. Για παράδειγμα, η κατηγορία μπορεί να περιλαμβάνει εικόνες με δεδομένο χρώμα ή σχήμα και αυτό θα είναι ορατό στις εικόνες βάρους.

## Ζήτημα 1.5:

In [ ]:
"""Regularised Linear Classifier"""


def one_hot(Y, num_classes):
  num_train = Y.size
  T = np. zeros((num_train, num_classes))
  T[np. arange(num_train), Y] = 1
  return T

lamda = 1.0

T_train = one_hot(Y_train, num_classes)

# we add the normalization term and this will help us to solve the linear system
#This constant is added on the diagonal of the array X_train*T_train
A = np.dot(X_train.T, X_train) + lamda * np.identity(num_dims)

# that is derived from the multiplication of two matrices, X_train^T and T_train for the array B
B = np. dot(X_train. T, T_train)

# We use the function np.linalg.solve to solve the linear system AW = B
W = np.linalg.solve(A, B)
"""
*****************************************************
"""

# compute accuracy on the test set

def linear_classify(X,W,Y):
  T_hat = np.dot(X,W)
  Y_hat = np.argmax(T_hat,1)
  accuracy = np.sum(Y_hat==Y)/np.size(Y)
  return Y_hat, accuracy

_,lin_accuracy=linear_classify(X_test,W,Y_test)

print('Linear classifier accuracy =% .2f%%' % (100.0*lin_accuracy))

# visualise the linear weights
im_util.plot_weights(W, cifar10_names)

## Ζήτημα 1.6:

In [ ]:
import numpy as np
import matplotlib. pyplot as plt


weights = W # The weights of the model
X_val = X_valid # Validation data
Y_val = Y_valid # Validation tags

#Define a list of other values of λ (lambda)
lambda_values = [0.001, 0.01, 0.1, 1, 10]

# We train a model for each value of λ
validation_accuracies = []
for lamda in lambda_values:
# Here we apply L2 normalization to the weights
 regularized_weights = weights / (1 + lamda)
# What is the accuracy on the validation set?
predictions = np. dot(X_val, regularized_weights)
accuracy = np.mean((predictions > 0.5) == Y_val)
validation_accuracies. append(accuracy)

# We select the λ that produces the highest accuracy
best_lambda = lambda_values[np. argmax(validation_accuracies)]
print(f"Best lambda: {best_lambda}")

# To assess the model, we apply it to the control set
test_predictions = np. dot(X_test, weights / (1 + best_lambda))
test_accuracy = np.mean((test_predictions > 0.5) == Y_test)
print(f"Test accuracy: {test_accuracy:. 4f}")

"Σχολιασμός της συμπεριφοράς του ταξινομητή καθώς μεταβάλετε το $\lambda$ "


Η διαδικασία κανονικοποίησης ενισχύεται όταν το λ αυξάνεται και αυτό κάνει το μοντέλο να δίνει μικρότερα βάρη και να το κάνει λιγότερο σύνθετο. Αυτό μπορεί να είναι χρήσιμο για την αντιμετώπιση του προβλήματος της υπερεκπαίδευσης των δικτύων, αλλά αν το λ γίνει μεγάλο τότε το δίκτυο γίνεται πιο απλό να εκπαιδεύσει και να δοκιμάσει άλλα δίκτυα

"Ενδεικτικά αποτελέσματα για διάφορες τιμές του lamda"

## Ζήτημα 1.7:

In [ ]:
"""Linear Classifier by Stochastic Gradient Descent"""

batch_size=32
weight_decay=0.01 # same as lambda
learning_rate=1e-2  # Play around with this hyperparameter until you improve validation accuracy.

num_epochs=10
num_iterations=num_epochs*(int)(num_train/batch_size)

np.random.seed(42)
W=np.random.randn(num_dims,num_classes)


valid_acc_seq=[]
iteration_seq=[]
W_seq=[]
W_sq_seq=[]

summary_interval=1000


for i in range(num_iterations):

    batch_indices = np.random.choice(num_train, batch_size)
    X_batch = X_train[batch_indices]
    Y_batch = Y_train[batch_indices]
    T_batch = one_hot(Y_batch, num_classes)

    # Calculate the gradient
    prediction = np.dot(X_batch, W)
    error = prediction - T_batch
    gradient = np.dot(X_batch.T, error) / batch_size + 2 * weight_decay * W

    W = W + dW

    if (i % summary_interval == 0):
        _,valid_acc=linear_classify(X_valid,W,Y_valid)
        valid_acc_seq.append(valid_acc)
        iteration_seq.append(i)
        print(' valid acc =% .2f%%' % (100.0 * valid_acc))
        W_seq.append(W)
        W_sq_seq.append(np.sum(W**2))

# plot validation accuracy and weight trends
plt.rcParams['figure.figsize'] = (16.0, 6.0)

fig=plt.figure()
plt.grid(True)
plt.plot(iteration_seq,valid_acc_seq,'r')
plt.xlabel('iteration')
plt.ylabel('accuracy')
plt.ylim(0,0.5)
plt.legend(['valid'])

fig=plt.figure()
plt.grid(True)
plt.plot(iteration_seq,np.log(W_sq_seq))
plt.xlabel('iteration')
plt.ylabel('log |W|^2')

# compute test accuracy
Y_hat,test_acc=linear_classify(X_test,W,Y_test)
print('\ntest accuracy = %.2f%%' % (100.0*test_acc))
im_util.plot_classification_examples(Y_hat,Y_test,im_test,cifar10_names)
im_util.plot_weights(W,cifar10_names)